<a href="https://colab.research.google.com/github/Afzal-Mansury/ANN_Bank_Customer_Churn_Prediction/blob/master/Optimizer_CNN_Model_Keras_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Creating CNN Model and Optimize it Using Keras **Tuner**

In [3]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 678kB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=e5bf7311c96e77cdae2687a4ca73ae22a7075ad5f695d144f4134f9d2b56df19
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=e0928bb91c22e09ac9bf4afea2ffca92f26e58c5ceca4561cbc945ba78666eac
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [5]:
print(tf.__version__)

2.3.0


Importing Data set from keras

In [6]:
fashion_mnist=keras.datasets.fashion_mnist

In [7]:
(train_images,train_labels),(test_images, test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


Scale Down the images

In [8]:
train_images = train_images/255.0
test_images = test_images/255.0

In [9]:
train_images[0].shape #checking shape of image

(28, 28)

In [10]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

Creating a function for building a model using hyperparameter tuning.
This will give which convolution layer I can use with filter 

In [11]:
def build_model(hp):
  model = keras.Sequential([keras.layers.Conv2D(filters=hp.Int("conv_1_filter",min_value=32, max_value=128, step=16),kernel_size = hp.Choice("conv_1_kernel",values = [3,5]),
                             activation ="relu",
                             input_shape=(28,28,1)
                            ),
                            keras.layers.Conv2D(
                                filters=hp.Int("conv_2_filter",min_value=32,max_value=64, step = 16),
                                kernel_size= hp.Choice("conv_2_kernel",values = [3,5]),
                                activation = "relu"
                            ),
                            keras.layers.Flatten(),
                            keras.layers.Dense(
                                units = hp.Int("dense_1_units",min_value=32, max_value=128, step=16),
                                activation = "relu"
                            ),
                            keras.layers.Dense(10,activation = "softmax")
  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice("learning_rate",values=[1e-2,1e-3])),
                loss="sparse_categorical_crossentropy",
                metrics=["accuracy"])
  
  return model

Trying to find best parameter for my model

In [12]:
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [13]:
tuner_search = RandomSearch(build_model, objective = "val_accuracy",max_trials=5,directory="output",project_name="Mnist Fashion")

In [14]:
tuner_search.search(train_images, train_labels, epochs=2, validation_split=0.1)

Epoch 1/2
1688/1688 [==============================] - 17s 10ms/step - loss: 0.5478 - accuracy: 0.8012 - val_loss: 0.4783 - val_accuracy: 0.8238
Epoch 2/2
1688/1688 [==============================] - 16s 9ms/step - loss: 0.4211 - accuracy: 0.8438 - val_loss: 0.3819 - val_accuracy: 0.8557


Epoch 1/2
1688/1688 [==============================] - 15s 9ms/step - loss: 0.4008 - accuracy: 0.8560 - val_loss: 0.3100 - val_accuracy: 0.8867
Epoch 2/2
1688/1688 [==============================] - 15s 9ms/step - loss: 0.2605 - accuracy: 0.9034 - val_loss: 0.2647 - val_accuracy: 0.9063


Epoch 1/2
1688/1688 [==============================] - 16s 9ms/step - loss: 0.4295 - accuracy: 0.8476 - val_loss: 0.3110 - val_accuracy: 0.8835
Epoch 2/2
1688/1688 [==============================] - 16s 10ms/step - loss: 0.2880 - accuracy: 0.8949 - val_loss: 0.2878 - val_accuracy: 0.8947


Epoch 1/2
1688/1688 [==============================] - 14s 8ms/step - loss: 0.3994 - accuracy: 0.8566 - val_loss: 0.3167 - val_accuracy: 0.8852
Epoch 2/2
1688/1688 [==============================] - 13s 8ms/step - loss: 0.2602 - accuracy: 0.9028 - val_loss: 0.2676 - val_accuracy: 0.8998


Epoch 1/2
1688/1688 [==============================] - 14s 8ms/step - loss: 0.4037 - accuracy: 0.8555 - val_loss: 0.3143 - val_accuracy: 0.8850
Epoch 2/2
1688/1688 [==============================] - 14s 8ms/step - loss: 0.2628 - accuracy: 0.9039 - val_loss: 0.2651 - val_accuracy: 0.9040


INFO:tensorflow:Oracle triggered exit


In [15]:
  model = tuner_search.get_best_models(num_models=1)[0]

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 80)        2080      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        34608     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 80)                1858640   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                810       
Total params: 1,896,138
Trainable params: 1,896,138
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.fit(train_images, train_labels, epochs = 10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.2034 - accuracy: 0.9239 - val_loss: 0.2537 - val_accuracy: 0.9105
Epoch 5/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.1573 - accuracy: 0.9405 - val_loss: 0.2651 - val_accuracy: 0.9097
Epoch 6/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.1209 - accuracy: 0.9553 - val_loss: 0.2746 - val_accuracy: 0.9127
Epoch 7/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0921 - accuracy: 0.9645 - val_loss: 0.3274 - val_accuracy: 0.9078
Epoch 8/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0704 - accuracy: 0.9740 - val_loss: 0.3406 - val_accuracy: 0.9110
Epoch 9/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0522 - accuracy: 0.9803 - val_loss: 0.4326 - val_accuracy: 0.9113
Epoch 10/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0444 - accuracy: 0.9838 - val_loss: 0.4674 - val_a

In [18]:
model.save("my_cnn_model_mnsit.h5")